# Sequence Models

## Recurrent Neural Networks

Examples in which sequence models such as recurrent neural networks (RNNs) are useful are the following:

+ Speech recognition: input is audio $\rightarrow$ output is a sequence of words
+ Music generation: input can be none or some parameters $\rightarrow$ output is a sequence of sounds
+ Sentiment classification: input is a comment $\rightarrow$ output is a score
+ DNA sequence analysis: inpute is a sequence of letters AGCCCCTGTGAAGGCTAG $\rightarrow$ output is detecting which part of the sequence corresponds to what
+ Machine translation: input is a sentence $\rightarrow$ output is a sentence
+ Video activity recognition: input is a sequece of frames $\rightarrow$ output is the recognized activity
+ Name entity recognition: input is a sentence $\rightarrow$ output are the people's names in it

### Notation

Consider a name entity recognition example.

The i-th input is $x^{(i)}$: "Harry Potter and Hermione Granger invented a new spell". Each element of the $x^{(i)}$ sample is denoted by $x^{(i)<t>}$, where $t = 1,2, ...T^{(i)}_x$. In this example $x^{(i)<1>}$ is "Harry".

The i-th target is $y^{(i)}:[1 \space 1 \space 0 \space 1 \space 1 \space 0 \space 0 \space 0 \space 0]$, where $y^{(i)<t>} = 1$ if the $t$ element is a person's name and 0 otherwise. The length of $y^{(i)}$ is denoted by $T^{(i)}_y$.

Notice that the length of each entry can differ, and also the the length of the input and the output can be different.

Consider a dictionary that takes the form of a vector whose elements are all the words admitted (generally around 50.000 words). 

Words can be represented as vectors of the same length of the dictionary with a 1 in the corresponding position of that word in the dictionary and zeros elsewhere.

### Recurrent Neural Network Model

#### Why not a standard network?

+ Inputs and outputs can have different lengths in different samples
+ It would not share features learned across different positions of text (if "Harry" is recognized as a word in a sample it should share this knowledge with other samples too) $\rightarrow$ this is the same thing with CNN with images
+ Each element (word) of each input (sentence) has the length of the dictionary, so it's very big

### Basic Recurrent Neural Network

In a one-directional RNN the information is processed from left to right and the parameters are shared:

<img src="images/RNN.png" width="800px" />

Starting from some vector of hidden units $a^{<0>}$ generally equal to zero, each element $x^{<t>}$ (a one-hot vector) is combined with some parameters $W_{ax}$ and $b_a$ and an activation function $g_1()$ (generally a $tanh$ function) to compute the hidden layer $a^{<t>} = g_1(W_{aa}a^{<t-1>} + W_{ax}x^{<t>}+b_a)$, this layer is then used to make a prediction $\hat{y}^{<t>} = g_2(W_{ya}a^{<t>}+b_y)$, where $g_2()$ is another activation function (maybe a sigmoid function if the problem is binary or softmax is the output has many classes).

<img src="images/description-block-rnn-ltr.png" width="600px" />


**The important thing is that the parameters $W_{aa}, W_{ax}, b_a, W_{ya}, b_y$ are the same across the whole sequence**. They will be updated through the backpropagation step.


So **each activation value $a^{<t-1>}$ is passed to the next one to make the following prediction**. However, one weakness of this RNN is that it only uses the information that is earlier in the sequence to make a prediction. In particular, when predicting $y^{<3>}$, it doesn't use information about the words $x^{<4>}$, $x^{<5>}$ and so on.

To simplify the notation let  $a^{<t>} = g(W_a [a^{<t-1>} , x^{<t>}]+b_a)$, where $[a^{<t-1>} , x^{<t>}]$ means stacking the two vectors one on top of the other. This way the matrix $W_a$ is the horizontal stacking of the matrices $[W_{aa};W_{ax}]$.

For example, if $a^{<t-1>}$ is a vector of length 100 and $x^{<t>}$ a vector of length 10.000, the new vector $[a^{<t-1>} , x^{<t>}]$ has length 10.100.




### Backpropagation

Consider again the forward propagation:
    
- Starting from some value of $a^{<0>}$, the initialized parameters $W_a,b_a$ and the first input $x^{<1>}$ we compute $a^{<1>}$
- We use $a^{<1>}$ together with initialized parameters $W_y,b_y$ to compute the probability $\hat{y}^{<1>}$
- We pass the same parameters $W_a,b_a$ and $W_y,b_y$ (together with $a^{<t-1>}$) to compute every $a^{<t>}$ and so $\hat{y}^{<t>}$

For every prediction $\hat{y}^{<t>}$ we compute the loss function $L^{<t>}(\hat{y}^{<t>} - y^{<t>}) = -y^{<t>}\log(\hat{y}^{<t>}) - (1-y^{<t>})\log(1-\hat{y}^{<t>})$, the typical logistic regression loss.

The overall loss is given by $L(\hat{y} - y) = \sum_{t=1}^{T_y} L^{<t>}(\hat{y}^{<t>} - y^{<t>})$

Backpropagation requires to do computations (passing messages) in the opposite direction to take derivatives with respect to the parameters $W_a, b_a, W_y, b_y$ in order to update them with gradient descent.

The most significant message to by passed is the one from $a^{<t>}$ to $a^{<t-1>}$, called **backpropagation through time**.

### Examples of sequence data and RNN architectures

|Type of RNN|Illustration|Example|
|-|-|-|
| One-to-one $$T_x = T_y = 1$$ |<img src="images/rnn-one-to-one-ltr.png" width="200px" /> |Traditional neural network|
|One-to-many $$T_x = 1, T_y > 1$$|<img src="images/rnn-one-to-many-ltr.png" width="400px" />|Music generation|
|Many-to-one $$T_x > 1, T_y = 1$$|<img src="images/rnn-many-to-one-ltr.png" width="400px" /> |Sentiment classification|
|Many-to-many $$T_x = T_y $$|<img src="images/rnn-many-to-many-same-ltr.png" width="400px" />|Name entity recognition|
|Many-to-many $$T_x \neq T_y $$ |<img src="images/rnn-many-to-many-different-ltr.png" width="400px" /> |Machine translation|


### Language Model and Sequence Generation

A language model estimates the probability of a sentence, being the probability of a sequence of words $P(y^{<1>}, y^{<2>}, ...,y^{<T_y>})$. An application is speach recognition where the input is an audio and the output is a sentence among all possible sentences.

To train a languange model you need a large corpus of text. Then you need to tokenize the text by mapping every word to a vector of zeros and a 1 to the corresponding position in the dictionary. Punctuation such as "." can also be tokenized as "end of sentence" $<EOS>$. Words not present in the dictionary are tokenized as "unknown" $<UNK>$.

Consider the example 

$$\text{Cats average 15 hours of sleep a day.}$$

The sequence generation hast a one-to-many structure in the form

<img src="images/rnn-one-to-many-ltr.png" width="400px" />

First we feed $a^{<0>} = 0$ and $x^{<0>} = 0$ to compute $a^{<1>}$ and $\hat{y}^{<1>}$, which is the probability of every word in the dictionary.

Going forward, we feed $y^{<1>}$ = "Cats" in the following computation of $a^{<2>}$ and $\hat{y}^{<2>}$, so that this time $\hat{y}^{<2>}$ is the probability of every word in the dictionary **conditional on** $y^{<1>}$ = "Cats" and so on. Notice that the probability of a sequence of 3 words is $P(y_1, y_2, y_3) = P(y_1)P(y_2|y_1)P(y_3|y_1, y_2)$.

The model works by minimizing the loss $L(\hat{y} - y) = \sum_{t=1}^{T_y} L^{<t>}(\hat{y}^{<t>} - y^{<t>})$ where for every word  $L^{<t>}(\hat{y}^{<t>} - y^{<t>}) = - \sum_{i=1}^M y_i^{<t>} log(\hat{y}_i^{<t>})$ where $M$ is the total number of feasible words and $y_j^{<t>}$ is the tokenized version of the word in position $j$ (a vector which has 1 in position $j$ and zeros elsewhere). This way if the true word is in position $j$ minimizing $- \sum_{i=1}^M y_i^{<t>} log(\hat{y}_i^{<t>})$ is equivalent to maximinzing $log(\hat{y}_j^{<t>})$, the probability of $y^{<t>} = j$.

### Vanishing gradients with RNNs

One problem of basic RNNs is that they're not very good in capturing long term dependence. For example, in the sentence

$$\text{The cat, which already [...], was full.}$$

the word "was" depends on the singular word "cat", which was much earlier in the sequence.

The **vanishing gradient problem**, typical of deep NNs means that it's difficult for the error computed at the end of the sequence on $\hat{y}^{<T_y>}$ to affects the computations that are earlier such as in $a^{<1>}$. On the contrary, there is much more influence on the closest words of the sequence.

Although less common, there may also be the problem of exploding gradients which makes parameters become very large and computation results in numerical overflow. 

A solution to that is called "gradient clipping" and consists in putting a cap to the maximum value for the gradient.

### Gated Recurrent Unit (GRU)

One way of dealing with vanishing gradients is to use **Gated Recurrent Units (GRU)** (instead of the normal hidden unit) which are based on the concept of gate $\Gamma$ and memory cell $c$:

$$\Gamma = \sigma(W x^{<t>} + U a^{<t-1>} + b)$$

where $\sigma$ is the sigmoid function which outputs a vector whose values are mostly very close to zero or one, and $W,U,b$ are parameters. The GRU will output an activation value equal to the memory cell $c^{<t>} = a^{<t>}$, which tells us how much to consider previous information.

In every GRU we will be considering overwriting the memory cell $c^{<t>}$ with a candidate 

$$\tilde{c}^{<t>} = \tanh(W_c[\Gamma_r * c^{<t-1>},x^{<t>}]+b_c)$$

where $\Gamma_r = \sigma(W_r[c^{<t-1>},x^{<t>}]+b_r)$ tells us if to drop previous infomation, the $r$ stands for "relevant".

We then use the candidate $\tilde{c}^{<t>}$ and another gate $\Gamma_u$, where $u$ stands for "update"

$$\Gamma_u = \sigma(W_u[c^{<t-1>},x^{<t>}]+b_u)$$

(vector with values mostly very close to zero or one) to update the elements of the memory cell $c^{<t>}$ if the corresponding element of $\Gamma_u$ is close to one

$$c^{<t>} = \Gamma_u * \tilde{c}^{<t>} + (1-\Gamma_u) * c^{<t-1>}$$

where $*$ is element-wise multiplication: notice that $c^{<t>}$ is a vector with same length of the hidden layer, so are $\tilde{c}^{<t>}$, $\Gamma_r$ and $\Gamma_u$.



In summary:

- starting from $a^{<t-1>} = c^{<t-1>}$ and $x^{<t>}$ compute the gates $\Gamma_r$ and $\Gamma_u$
- with the gate $\Gamma_r$ compute the candidate $\tilde{c}^{<t>}$
- use $\Gamma_u$ and the candidate $\tilde{c}^{<t>}$ to eventually update $c^{<t>}$
- pass the new $c^{<t>} = a^{<t>}$ to the next unit


<img src="images/gru-ltr.png" width="400px" />

For example, in the sentence

$$\text{The cat, which already ate [...], was full}$$

it's optimal to update the memory cell at "cat" ($\Gamma_u = 1$), then keeping $\Gamma_u = 0$ for the consecuitve words until reaching the word "was". After "was", there is no need to memorize the word "cat" anymore, so we can set again $\Gamma_u = 1$. What these element-wise multiplications do is it just tells you GRU which are the dimensions of your memory cell vector to update at every time step. You can choose to keep some bits constant while updating other bits. For example, maybe you'll use one-bit to remember the singular or plural cat, and maybe you'll use some other bits to realize that you're talking about food. Because we talked about eating and talk about foods, then you'd expect to talk about whether the cat is full later. You can use different bits and change only a subset of the bits at every point in time.



### Long Short Term Memory (LSTM)

Another way of dealing with vanishing gradients is the **Long Short Term Memory (LSTM)** unit, even more powerful than GRU.

In LSTM the candidate memory cell and the update gate are computed in the same way of GRU but this time $a^{<t>}$ is different from $c^{<t>}$

$$\tilde{c}^{<t>} = \tanh(W_c[\Gamma_r * a^{<t-1>},x^{<t>}]+b_c)$$

$$\Gamma_u = \sigma(W_u[a^{<t-1>},x^{<t>}]+b_u)$$


The memory cell $c^{<t>}$ is update not only through $\Gamma_u$ but also through a "forget" gate $\Gamma_f$

$$\Gamma_f = \sigma(W_f[a^{<t-1>},x^{<t>}]+b_f))$$

$$c^{<t>} = \Gamma_u * \tilde{c}^{<t>} + \Gamma_f * c^{<t-1>}$$

This gives the cell the option of keeping old values ($c^{<t-1>}$) and adding them to the candidate ($\tilde{c}^{<t>}$)

The new activation value is computed using a fourth "output" gate

$$\Gamma_o = \sigma(W_o[a^{<t-1>},x^{<t>}]+b_o))$$

such that

$$a^{<t>} = \Gamma_o * \tanh(c^{<t>})$$

where $*$ is element-wise multiplication.

In summary,

- You use $a^{<t-1>}$ and $x^{<t>}$ to compute the gates $\Gamma_f$, $\Gamma_u$ and $\Gamma_o$ and the candidate $\tilde{c}^{<t>}$ (the "relevant" gate $\Gamma_r$ is omitted)
- You compute $c^{<t>}$
- You output $a^{<t>}$

<img src="images/lstm_unit.jpg" width="400px" />

By stacking many LSTM units one after the other it is relatively easy that the memory cell $c^{<t>}$ carries information of many previous memory cells.

In a variation of LSTM the gates depend also on the previous memory cell value $c^{<t-1>}$, this is called "peephole connection".

In the history of Deep Learning LSTN were invented before, and GRU were derived as simplifications of the more complicated LSTM model. There isn't a superior algorithm between the two. However, since GRU are a simpler model (only two gates) it is easier to build a bigger network.

### Bidirectional RNN

No matter if the units are standard RNN, GRU or LSTM, it's still be difficult to predict a word without knowing the following part of the sentence. For example estimating if the word "Teddy" refers to a name in the two sentences:

$$\text{Teddy bears are on sale!}$$
$$\text{Teddy Roosevelt was a great President}$$

In Bidirectional RNN in addition to activations going forward $\vec{a}^{<t>}$ where each one is pass to the next unit, there are other activations starting from the end and going backward $\overleftarrow{a}^{<t>}$:

<img src="images/brnn.jpg" width="600px" />

So the prediction is made by

$$\hat{y}^{<t>} = g(W_y[\vec{a}^{<t>},\overleftarrow{a}^{<t>}]+b_y)$$

This way the prediction of $y^{<3>}$ depends both on $x^{<1>}$ through $\vec{a}^{<1>}, \vec{a}^{<2>}, \vec{a}^{<3>}$, but also on $x^{<4>}$ through $\overleftarrow{a}^{<4>}$ and $\overleftarrow{a}^{<3>}$, taking also information from the future.

It seems to be common to use BRNN with LSTN units. The disadvantage is that you need to go through the whole sequence of data before making a prediction. In speak recognition it would need to wait for the end of the speach before processing it.

### Deep RNNs

For learning very difficult functions sometimes it's useful to stack many layers together. Instead of having only one activation layer $a^{<t>}$ between each $x^{<t>}$ and $y^{<t>}$ in Deep RNNs we stack multiples hidden layers on top of each other:

<img src="images/deep-rnn-ltr.png" width="400px" />


This way each hidden layer denoted by $a^{[l]<t>}$ is computed as 

$$a^{[l]<t>} = g(W_a^{[l]}[a^{[l]<t-1>},a^{[l]<t>}]+b_a^{[l]}$$

where the parameters $W_a^{[l]}$ and $b_a^{[l]}$ are shared horizontally on the layer.

For RNNs having three layers is already a lot because of computation. However, there are versions of DRNNs where on top of three RNNs layers instead of outputting directly $\hat{y}^{<t>}$ there are many other layer not connected horizontally between the last $a^{[k]<t>}$ and the prediction.

Consider instead an input image dark-to-light, with columns $[0,...0,10...10]$, applying the convlution would result in an image gray-dark-gray, with colummns $[0,-30,0]$. To solve this issue generally is applied the absolute value.

An horizontal filter would be made of rows $$\left[\begin{array}{ccc}1 & 1 & 1\\0 & 0 & 0\\-1 & -1 & -1\end{array}\right]$$

Applying Deep Learning means that we don't need to handcraft these numbers, we can treat them as weights and then learn them. It can learn horizontal, vertical, angled, or any edge type automatically rather than getting them by hand:

$$\left[\begin{array}{ccc}w_1 & w_2 & w_3\\w_4 & w_5 & w_6\\w_7 & w_8 & w_9\end{array}\right]$$

Strided convolutions refers to fix a number $s$ to define the number of pixels the algorithm will jump when applying the filter. A stride of $s=2$ means that the filter will cover the input matrix moving by $2$ cells each time.

The resulting matrix has dimension

$$\bigg(\frac{(n+2p-f)}{s}+1\bigg) \times \bigg(\frac{(n+2p-f)}{s}+1\bigg)$$

If the dimension is not made of integers it is rounded down using the `floor()` function, denoted by $\lfloor \dots \rfloor$. 

In math textbooks the convolution operation flips the filter before applying it to the imput matrix:

$$\left[\begin{array}{ccc}w_1 & w_2 & w_3\\w_4 & w_5 & w_6\\w_7 & w_8 & w_9\end{array}\right] \rightarrow \left[\begin{array}{ccc}w_9 & w_8 & w_7\\w_6 & w_5 & w_4\\w_3 & w_2 & w_1\end{array}\right]$$

But in DL there is no flipping. It is still referred to as convolution even if it would be a cross-correlation.

It is possible to detect horizontal edges only for a channel and keep the others equal to zero:

$$\underbrace{\left[\begin{array}{ccc}1 & 1 & 1\\0 & 0 & 0\\-1 & -1 & -1\end{array}\right]}_R \quad \underbrace{\left[\begin{array}{ccc}0 & 0 & 0\\0 & 0 & 0\\0 & 0 & 0\end{array}\right]}_G\quad \underbrace{\left[\begin{array}{ccc}0 & 0 & 0\\0 & 0 & 0\\0 & 0 & 0\end{array}\right]}_B$$

It is possible to use multiple filters at the same time, for example one vertical and one horizontal edge detector. The two outputs can be stacked together with depth equal to the numbe of filter used, for example:

<img src="images/mult_filters.png" width="600px" />

$$
(6\times6\times3) \text{ input image} \rightarrow 
 \biggl\{\begin{array}{c}(3\times3\times3)\text{ "vertical" filter} \rightarrow (4\times4) \text{ matrix}\\
(3\times3\times3)\text{ "horizontal" filter} \rightarrow (4\times4) \text{ matrix}\end{array} \biggr\} \rightarrow
(4\times4\times2) \text{ output}
$$

Instead of what happens in a **plain network**:

$a^{[l]} ==> \underbrace{z^{[l+1]} = W^{[l+1]} a^{[l]} + b^{[l+1]}}_{\text{linear}} ==> \underbrace{a^{[l+1]} = g(z{[l+1]})}_{\text{ReLU}} ==> \underbrace{z{[l+2]} = W{[l+2]} a{[l+1]} + b{[l+2]}}_{\text{linear}} ==> \underbrace{a{[l+2]} = g(z{[l+2]})}_{\text{ReLU}}$

We take a *short cut* / skip connection to make it a **residual network**:

$a^{[l]} ==> ... ==> \underbrace{a{[l+2]} = g(z{[l+2]} + a^{[l]})}_{\text{ReLU}}$

This is done after the last linear operation and before the last ReLU operation and forms a **residual block**, which allows to train much deeper NNs.

<img src="images/res_block.png" width="1000px" />

A second version called MobileNet v2 use a "bottleneck block" with a residual connection repeated 17 times instead of the simpler depthwise separable convolution.

<img src="images/MobileNets.png" width="600px" />

This version was introduced in [Sandler et al. 2019, MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381)

Object localization is about defining a box around the predicted label ("classification with localization"), it is the step before object detection (where there can by multiple objects of different classes).

<img src="images/object_det.jpeg" width="600px"/>

The predictionis not only about the class $1, ...C$ , but also about the box in which the class is:

$$
y_i = \left[\begin{array}{c}
p_c \in \{0,1\} & \text{is there any object?} \\
b_x & \text{x-axis center of the box} \\
b_y & \text{y-axis center of the box} \\
b_h & \text{height of the box} \\
b_w & \text{width of the box} \\
c_1 \in \{0,1\} & \text{class 1 object} \\
... & \text{...} \\
c_C \in \{0,1\} & \text{class C object}
\end{array}\right]
$$

The dimension of the boxes goes by convention from N-W: $\{0,0\}$ to S-E: $\{1,1\}$.

If there is no object then $p_c = 0$ and the rest of the vector $y_i$ is irrilevant.